## Importación de librerías

In [43]:
import pandas as pd
import os
import glob
import csv
import json

## Configuración de variables globales

In [44]:
with open('setup.json', 'r') as f:
    SETUP_JSON = json.load(f)
NA_VALUES = SETUP_JSON['navalues'] # Consistent NA values,
CHUNK_SIZE = SETUP_JSON['chunk_size'] # Chunk size for reading CSV files,
SAMPLE_FRACTION = SETUP_JSON['sample_fraction'] # Fraction of data to sample,
OUTPUT_CSV = SETUP_JSON['output_csv'] # Output CSV file
OUTPUT_PARQUET = SETUP_JSON['output_parquet'] # Output CSV file
MAX_COLUMNS = SETUP_JSON['max_columns'] # Maximum number of columns to display

## Combinar datasets

Unimos todos los archivos CSV y guardamos el dataset en formato CSV (para analizarlo con Tableau) y en formato Parquet (para su tratamiento con Python).

In [45]:

def find_datasets_with_same_header():
    headers = {}
    same_files = {}

    for file in DATASETS:
        with open(file, newline='', encoding='utf-8') as f:
            header = tuple(next(csv.reader(f), None))
            if header:
                if header in headers:
                    headers[header].append("datasets\\"+file)
                else:
                    headers[header] = ["datasets\\"+file]
    print("Archivos con headers distintos:", tuple(headers.values())[1])
    return tuple(headers.values())[0]

In [46]:
csv_files = find_datasets_with_same_header()
print("Archivos con el mismo encabezado:", csv_files)

Archivos con headers distintos: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-20-2018.csv']
Archivos con el mismo encabezado: ['datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-14-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-15-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-16-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-21-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-22-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-23-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\02-28-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traffic-network-classify\\datasets\\03-01-2018.csv', 'datasets\\c:\\Users\\isard\\Desktop\\AI-for-Traff

In [47]:
import pandas as pd


def contar_y_listar_valores_unicos(df, columna_labels):
    """
    Cuenta y lista los valores únicos en la columna especificada de un DataFrame.

    Args:
        df (pd.DataFrame): El DataFrame que contiene la columna.
        columna_labels (str): El nombre de la columna a analizar.

    Returns:
        tuple: Número de valores únicos y una lista con los valores únicos.
    """
    valores_unicos = df[columna_labels].unique()
    print(f"Número de valores únicos en la columna '{columna_labels}': {len(valores_unicos)}")
    # return len(valores_unicos), valores_unicos
    return valores_unicos


def merge_csv_files(file_list, output_csv, output_parquet):
    merged_df = pd.DataFrame()

    for file in file_list:
        df = pd.read_csv(file, nrows=10001)  # Leer encabezado + 10000 líneas
        merged_df = pd.concat([merged_df, df.iloc[1:]])  # Excluir el encabezado después del primer archivo
    valores_unicos = contar_y_listar_valores_unicos(merged_df, 'Label')
    print("Valores únicos en la columna 'Label':", valores_unicos)
    valores_unicos = {label: index for index, label in enumerate(valores_unicos)}
    merged_df['Label_index'] = merged_df['Label'].map(valores_unicos)
    
    
    # Guardar en CSV
    merged_df.to_csv(output_csv, index=False)
    print(f"Archivo CSV guardado como {output_csv}")

    # Guardar en Parquet
    merged_df.to_parquet(output_parquet, index=False)
    print(f"Archivo Parquet guardado como {output_parquet}")
    return merged_df

file_list = ["datasets\\02-14-2018.csv", "datasets\\02-15-2018.csv", "datasets\\02-16-2018.csv", "datasets\\02-21-2018.csv", "datasets\\02-22-2018.csv"]
df = merge_csv_files(file_list, OUTPUT_CSV, OUTPUT_PARQUET)

Número de valores únicos en la columna 'Label': 9
Valores únicos en la columna 'Label': ['Benign' 'FTP-BruteForce' 'DoS attacks-GoldenEye'
 'DoS attacks-SlowHTTPTest' 'DDOS attack-LOIC-UDP' 'DDOS attack-HOIC'
 'Brute Force -Web' 'Brute Force -XSS' 'SQL Injection']
Archivo CSV guardado como merged_output.csv
Archivo Parquet guardado como merged_output.parquet


In [51]:
import pandas as pd

# Revisar cada columna y contar los tipos de valores
for col in df.columns:
    tipos = df[col].apply(type).value_counts()
    if len(tipos) > 1:  # Si hay más de un tipo, lo imprimimos
        print(f"Columna: {col}")
        print(tipos)
        print("-" * 30)
    else:
        print(f"Columna: {col} tiene un solo tipo de dato: {tipos.index[0]}")
        print("-" * 30)


Columna: Dst Port tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Protocol tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Timestamp tiene un solo tipo de dato: <class 'str'>
------------------------------
Columna: Flow Duration tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Tot Fwd Pkts tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Tot Bwd Pkts tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: TotLen Fwd Pkts tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: TotLen Bwd Pkts tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Fwd Pkt Len Max tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Fwd Pkt Len Min tiene un solo tipo de dato: <class 'int'>
------------------------------
Columna: Fwd Pkt Len Mean tiene un solo tipo de 

In [50]:
df.head()

,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,Label_index
1,0,0,14/02/2018 08:33:50,112641466,3,0,0,0,0,0,...,0.0,0.0,0,0,56320733.0,114.551299,56320814,56320652,Benign,0
2,0,0,14/02/2018 08:36:39,112638623,3,0,0,0,0,0,...,0.0,0.0,0,0,56319311.5,301.934596,56319525,56319098,Benign,0
3,22,6,14/02/2018 08:40:13,6453966,15,10,1239,2273,744,0,...,0.0,0.0,0,0,0.0,0.000000,0,0,Benign,0
4,22,6,14/02/2018 08:40:23,8804066,14,11,1143,2209,744,0,...,0.0,0.0,0,0,0.0,0.000000,0,0,Benign,0
5,22,6,14/02/2018 08:40:31,6989341,16,12,1239,2273,744,0,...,0.0,0.0,0,0,0.0,0.000000,0,0,Benign,0
